In [ ]:
!pip install pdfreader --quiet
!pip install tabula-py --quiet
!pip install JPype1
!pip install PyPDF2

In [ ]:
import os
os.environ["JAVA_HOME"] = "C:/Program Files (x86)/Common Files/Oracle/Java/javapath"

In [ ]:
#gerar as listas de acordo com o numero de páginas

import os
from PyPDF2 import PdfReader

# Caminho do diretório
caminho_diretorio = r'G:\Meu Drive\Portifólio GitHub\malaria-data-analysis\etl\example_pdfs'

# Lista todos os arquivos no diretório
arquivos = os.listdir(caminho_diretorio)

# Listas para armazenar arquivos com uma página e mais de uma página
global arquivos_uma_pagina
arquivos_uma_pagina = []
arquivos_mais_de_uma_pagina = []

# Filtra e verifica o número de páginas nos arquivos PDF
for nome_arquivo in arquivos:
    if nome_arquivo.endswith('.pdf'):
        caminho_arquivo = os.path.join(caminho_diretorio, nome_arquivo)
        with open(caminho_arquivo, 'rb') as arquivo:
            leitor = PdfReader(arquivo)
            num_paginas = len(leitor.pages)
            if num_paginas == 1:
                arquivos_uma_pagina.append(nome_arquivo)
            else:
                arquivos_mais_de_uma_pagina.append(nome_arquivo)

# Imprime os arquivos PDF com uma página e mais de uma página
print("Arquivos com uma página:")
for arquivo in arquivos_uma_pagina:
    print(arquivo)

print("\nArquivos com mais de uma página:")
for arquivo in arquivos_mais_de_uma_pagina:
    print(arquivo)


#PROCESSA OS ARQUIVOS DE MAIS DE UMA PÁGINA
#gerar os arquivos de mais de uma página

import tabula
import pandas as pd
import re
from PyPDF2 import PdfReader

# Caminho do diretório
caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)


# Converte 7,5 cm para pontos (1 cm = 28,35 pontos)
topo = 7.5 * 28.35

# Ajusta o topo para começar duas linhas acima
topo_ajustado = topo - 1 * 28.35

# Itera sobre a lista de arquivos com mais de uma página
for nome_arquivo in arquivos_mais_de_uma_pagina:
    caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)

    # Lê a primeira página do arquivo PDF como uma lista de DataFrames do Pandas
    # Especifica a área da página a ser analisada
    lista_df = tabula.read_pdf(caminho_do_arquivo, pages=1, area=[topo_ajustado, 0, 1000, 1000], pandas_options={'header': None})

    # Seleciona o primeiro DataFrame da lista
    df_pagina1 = lista_df[0]

    df_pagina1.columns = ['País Infecção', 'Município de Infecção', 'Localidade de Infecção', 'Positivos', 'F', 'V', 'F+V', 'M', 'O', 'Não F']

    # Lê todas as páginas do arquivo PDF como uma lista de DataFrames do Pandas
    lista_df = tabula.read_pdf(caminho_do_arquivo, pages='all', pandas_options={'header': None})

    # DataFrame para as páginas restantes
    df_resto = pd.DataFrame()

    for i, df in enumerate(lista_df[1:]):
        num_colunas = len(df.columns)
        if num_colunas == 9:
            df.insert(2, 'Localidade de Infecção', '')
        df.columns = ['País Infecção', 'Município de Infecção', 'Localidade de Infecção', 'Positivos', 'F', 'V', 'F+V', 'M', 'O', 'Não F']
        df_resto = pd.concat([df_resto, df])

    # Une os dois DataFrames
    df_total = pd.concat([df_pagina1, df_resto])

    # Lê o cabeçalho do arquivo PDF para extrair UF, MUNICÍPIO e Período
    with open(caminho_do_arquivo, 'rb') as arquivo:
        leitor = PdfReader(arquivo)
        primeira_pagina = leitor.pages[0]
        texto = primeira_pagina.extract_text()
        uf, municipio, periodo = re.search(r'UF: (.+) MUNICÍPIO: (.+) Período: (.+)', texto).groups()

    # Adiciona as colunas UF, MUNICÍPIO e Período
    df_total['UF'] = uf
    df_total['MUNICÍPIO'] = municipio
    df_total['Período'] = periodo

    # Remove a última linha do DataFrame final
    df_total = df_total.iloc[:-1]

    # Exporta o DataFrame total como um arquivo Excel
    caminho_do_arquivo_xlsx = caminho_do_arquivo.replace('.pdf', '.xlsx')
    df_total.to_excel(caminho_do_arquivo_xlsx, index=False)

# TENTA EXTRAIR AUTOMATICAMENTE CADO A CASO, QUADRO A QUADRO

import os
from PyPDF2 import PdfReader
import tabula
import pandas as pd
import re

# Lista de arquivos
lista_arquivos = arquivos_uma_pagina

# Converte 7,5 cm para pontos (1 cm = 28,35 pontos)
topo = 7.5 * 28.35

# Ajusta o topo para começar duas linhas acima
topo_ajustado = topo - 1 * 28.35

# Itera sobre a lista de arquivos
for nome_arquivo in lista_arquivos:
    # Flag para verificar se o arquivo foi processado com sucesso
    processado_com_sucesso = False

    # Itera sobre a altura da área de interesse
    for altura in range(150, 1000, 25):
        try:
            # Caminho do arquivo
            caminho_arquivo = os.path.join(caminho_diretorio, nome_arquivo)

            # Lendo a tabela do arquivo PDF
            tabelas = tabula.read_pdf(caminho_arquivo, pages='all', area=[topo_ajustado, 0, altura, 1000], pandas_options={'header': None})

            # Supondo que a tabela que você quer é a primeira
            tabela = tabelas[0]

            # Encontra a linha que contém 'Total' e remove todas as linhas depois dela
            total_index = tabela[tabela[0] == 'Total'].index[0]
            tabela = tabela.iloc[:total_index]

            # Verificando o número de colunas
            num_colunas = len(tabela.columns)
            if num_colunas == 9:
                # Adicionando a coluna 'Localidade de Infecção' com valores em branco
                tabela.insert(2, 'Localidade de Infecção', '')

            # Renomeando as colunas
            tabela.columns = ['País Infecção', 'Município de Infecção', 'Localidade de Infecção', 'Positivos', 'F', 'V', 'F+V', 'M', 'O', 'Não F']

            # Lendo o cabeçalho do arquivo PDF para extrair UF, MUNICÍPIO e Período
            with open(caminho_arquivo, 'rb') as arquivo:
                leitor = PdfReader(arquivo)
                primeira_pagina = leitor.pages[0]
                texto = primeira_pagina.extract_text()
                uf, municipio, periodo = re.search(r'UF: (.+) MUNICÍPIO: (.+) Período: (.+)', texto).groups()

            # Adicionando as colunas UF, MUNICÍPIO e Período
            tabela['UF'] = uf
            tabela['MUNICÍPIO'] = municipio
            tabela['Período'] = periodo

            # Caminho do arquivo de saída
            caminho_saida = caminho_arquivo.replace('.pdf', f'_verificar_{altura}.xlsx')

            # Exportando a tabela para um arquivo Excel
            tabela.to_excel(caminho_saida, index=False)

            # Se chegamos até aqui, o arquivo foi processado com sucesso
            processado_com_sucesso = True

            # Interrompe o loop da altura
            break
        except Exception as e:
            print(f"Erro ao processar o arquivo {nome_arquivo} com altura {altura}: {e}")

    # Se o arquivo não foi processado com sucesso, imprime uma mensagem
    if not processado_com_sucesso:
        print(f"Não foi possível processar o arquivo {nome_arquivo} com nenhuma das alturas especificadas.")

In [ ]:
import tabula
import pandas as pd
import re
from PyPDF2 import PdfReader

# Caminho do diretório
caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)

# Converte 7,5 cm para pontos (1 cm = 28,35 pontos)
topo = 7.5 * 28.35

# Ajusta o topo para começar duas linhas acima
topo_ajustado = topo - 1 * 28.35

# Função para processar a primeira página
def processar_pagina1(nome_arquivo):
    caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)
    lista_df = tabula.read_pdf(caminho_do_arquivo, pages=1, area=[topo_ajustado, 0, 1000, 1000], pandas_options={'header': None})
    df_pagina1 = lista_df[0]
    df_pagina1.columns = ['País Infecção', 'Município de Infecção', 'Localidade de Infecção', 'Positivos', 'F', 'V', 'F+V', 'M', 'O', 'Não F']
    return df_pagina1

# Função para exportar os dados
def exportar_dados(df, nome_arquivo):
    caminho_do_arquivo_xlsx = nome_arquivo.replace('.pdf', '.xlsx')
    df.to_excel(caminho_do_arquivo_xlsx, index=False)

# Itera sobre a lista de arquivos com mais de uma página
for nome_arquivo in arquivos_mais_de_uma_pagina:
    df_pagina1 = processar_pagina1(nome_arquivo)

    # Exporta o DataFrame da primeira página como um arquivo Excel
    exportar_dados(df_pagina1, nome_arquivo)

In [ ]:
import tabula
import pandas as pd
import re
from PyPDF2 import PdfReader

# Caminho do diretório
caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)

# Converte 7,5 cm para pontos (1 cm = 28,35 pontos)
topo = 7.5 * 28.35

# Ajusta o topo para começar duas linhas acima
topo_ajustado = topo - 1 * 28.35

# Função para processar as demais páginas
def processar_demais_paginas(nome_arquivo):
    caminho_do_arquivo = os.path.join(caminho_diretorio, nome_arquivo)
    lista_df = tabula.read_pdf(caminho_do_arquivo, pages='all', pandas_options={'header': None})
    df_resto = pd.DataFrame()
    for i, df in enumerate(lista_df[1:]):
        num_colunas = len(df.columns)
        if num_colunas == 9:
            df.insert(2, 'Localidade de Infecção', '')
        df.columns = ['País Infecção', 'Município de Infecção', 'Localidade de Infecção', 'Positivos', 'F', 'V', 'F+V', 'M', 'O', 'Não F']
        df_resto = pd.concat([df_resto, df])
    return df_resto

# Função para exportar os dados
def exportar_dados(df, nome_arquivo, sufixo):
    caminho_do_arquivo_xlsx = nome_arquivo.replace('.pdf', f'_{sufixo}.xlsx')
    df.to_excel(caminho_do_arquivo_xlsx, index=False)

# Itera sobre a lista de arquivos com mais de uma página
for nome_arquivo in arquivos_mais_de_uma_pagina:
    df_resto = processar_demais_paginas(nome_arquivo)

    # Exporta o DataFrame das demais páginas como um arquivo Excel
    exportar_dados(df_resto, nome_arquivo, 'demais_paginas')